In [1]:
import knoty
import k3d
import sympy as sp
import numpy as np

In [2]:
plot = k3d.plot()
plot = knoty.plot_contact_structure(plot, alpha = 1)

plot.display()

Output()

In [3]:
# Example usage with your basic_knot function
def knot1(t):
    return 3 * sp.sin(2 * np.pi *t) * sp.cos(2 * np.pi *t), sp.cos(2 * np.pi *t), sp.sin(2 * np.pi *t)**3

def knot2(t):
    return sp.cos(2 * np.pi *t), sp.sin(2 * np.pi *2*t), 2/3 * sp.sin(2 * np.pi *t)*sp.cos(2 * np.pi *2*t) -4/3 * sp.sin(2 * np.pi *2*t)*sp.cos(2 * np.pi *t)


The following knot equations were kindly provided by Lenny Ng

In [4]:

def LNGLegTrefoil(t):
    # Adjusted definitions with the correct parameter shifts as in the Mathematica version
    LegTref1_x = 3 * t
    LegTref1_y = 3/32 * (4 + 5 * t * (sp.sqrt(1 - t) - sp.sqrt(1 + t)) - 
                         3 * t**3 * (sp.sqrt(1 - t) - sp.sqrt(1 + t)) + 
                         2 * t**2 * (-2 + sp.sqrt(1 - t) + sp.sqrt(1 + t))) * \
                (-4 + ((1 - (1 - t)**(sp.Rational(3, 2))) * (2 + 3 * t - t**3) + (2 - 3 * t + t**3) * 
                (-1 + (1 + t)**(sp.Rational(3, 2))))**2)
    LegTref1_z = 1/16 * ((1 - (1 - t)**(sp.Rational(3, 2))) * (2 + 3 * t - t**3) + (2 - 3 * t + t**3) * 
                (-1 + (1 + t)**(sp.Rational(3, 2)))) * \
                (-12 + ((1 - (1 - t)**(sp.Rational(3, 2))) * (2 + 3 * t - t**3) + (2 - 3 * t + t**3) * 
                (-1 + (1 + t)**(sp.Rational(3, 2))))**2)

    LegTref3_y = -(9/4) * t * sp.sqrt(1 - t**2)
    LegTref3_z = 1 + 9/4 * (1 - t**2)**(sp.Rational(3, 2))

    # Using updated expressions with parameter shifts
    x = 3 * t
    x = sp.Piecewise((LegTref1_x.subs(t, t - 1), t <= 2), 
                     (LegTref1_x.subs(t, 3 - t), (t > 2) & (t <= 4)), 
                     (LegTref1_x.subs(t, t - 5), (t > 4) & (t <= 6)), 
                     (LegTref1_x.subs(t, 7 - t), t > 6))
    y = sp.Piecewise((LegTref1_y.subs(t, t - 1), t <= 2), 
                     (LegTref3_y.subs(t, 3 - t), (t > 2) & (t <= 4)), 
                     (-LegTref1_y.subs(t, t - 5), (t > 4) & (t <= 6)), 
                     (-LegTref3_y.subs(t, 7 - t), t > 6))
    z = sp.Piecewise((LegTref1_z.subs(t, t - 1), t <= 2), 
                     (LegTref3_z.subs(t, 3 - t), (t > 2) & (t <= 4)), 
                     (-LegTref1_z.subs(t, t - 5), (t > 4) & (t <= 6)), 
                     (-LegTref3_z.subs(t, 7 - t), t > 6))

    return x, y, z

def LNGLegLHTrefoil(t):
    LegLHtref1_x = t
    LegLHtref1_y = 3/2 * (sp.sqrt(t) - 4 * (-1 + sp.sqrt(1 - t)) * t + (-4 + 11 * sp.sqrt(1 - t)) * t**2 - 7 * t**(sp.Rational(5, 2)) - 6 * sp.sqrt(1 - t) * t**3 + 6 * t**(sp.Rational(7, 2)))
    LegLHtref1_z = (1 - (1 - t)**(sp.Rational(3, 2))) * (3 * t**2 - 2 * t**3) + t**(sp.Rational(3, 2)) * (1 - 3 * t**2 + 2 * t**3)

    LegLHtref3_x = 2 - t
    LegLHtref3_y = -1.125 + 8.25 * sp.sqrt(3 - t) + 2.25 * sp.sqrt(-1 + t) + (1.5 - 14.625 * sp.sqrt(3 - t) - 8.625 * sp.sqrt(-1 + t)) * t + (-0.375 + 7.5 * sp.sqrt(3 - t) + 6 * sp.sqrt(-1 + t)) * t**2 + (-1.125 * sp.sqrt(3 - t) - 1.125 * sp.sqrt(-1 + t)) * t**3
    LegLHtref3_z = 1/4 * (-(0.5 + (3 - t)**(sp.Rational(3, 2))) * (-4 + t) * (-1 + t)**2 + (-3 + t)**2 * t * (1 - sp.sqrt(-1 + t) + sp.sqrt(-1 + t) * t))

    LegLHtref5_x = -4 + t
    LegLHtref5_y = 5.625 + 76.5 * sp.sqrt(5 - t) + 52.5 * sp.sqrt(-3 + t) + (-3 - 58.125 * sp.sqrt(5 - t) - 46.125 * sp.sqrt(-3 + t)) * t + (0.375 + 14.25 * sp.sqrt(5 - t) + 12.75 * sp.sqrt(-3 + t)) * t**2 + (-1.125 * sp.sqrt(5 - t) - 1.125 * sp.sqrt(-3 + t)) * t**3
    LegLHtref5_z = 1/4 * ((5 - t)**(sp.Rational(3, 2)) * (-6 + t) * (-3 + t)**2 + (0.5 - (-3 + t)**(sp.Rational(3, 2))) * (-5 + t)**2 * (-2 + t))

    LegLHtref6_x = 6 - t
    LegLHtref6_y = 3/2 * (120 - 1045 * sp.sqrt(6 - t) + 924 * sp.sqrt(-5 + t) + (-44 + 564 * sp.sqrt(6 - t) - 520 * sp.sqrt(-5 + t)) * t + (4 - 101 * sp.sqrt(6 - t) + 97 * sp.sqrt(-5 + t)) * t**2 + 6 * (sp.sqrt(6 - t) - sp.sqrt(-5 + t)) * t**3)
    LegLHtref6_z = -(1 - (6 - t)**(sp.Rational(3, 2))) * (-5 + t)**2 * (-13 + 2 * t) + (-6 + t)**2 * (-5 + t)**(sp.Rational(3, 2)) * (-9 + 2 * t)

    LegLHtref8_x = -6 + t
    LegLHtref8_y = 18 - 409.5 * sp.sqrt(8 - t) - 336 * sp.sqrt(-6 + t) + (-5.25 + 174 * sp.sqrt(8 - t) + 153 * sp.sqrt(-6 + t)) * t + (0.375 - 24.375 * sp.sqrt(8 - t) - 22.875 * sp.sqrt(-6 + t)) * t**2 + (1.125 * sp.sqrt(8 - t) + 1.125 * sp.sqrt(-6 + t)) * t**3
    LegLHtref8_z = 1/4 * (-(0.5 + (8 - t)**(sp.Rational(3, 2))) * (-9 + t) * (-6 + t)**2 + (1 + (-6 + t)**(sp.Rational(3, 2))) * (-8 + t)**2 * (-5 + t))

    LegLHtref10_x = 10 - t
    LegLHtref10_y = -30 - 864 * sp.sqrt(10 - t) - 742.5 * sp.sqrt(-8 + t) + (6.75 + 285 * sp.sqrt(10 - t) + 258 * sp.sqrt(-8 + t)) * t + (-0.375 - 31.125 * sp.sqrt(10 - t) - 29.625 * sp.sqrt(-8 + t)) * t**2 + (1.125 * sp.sqrt(10 - t) + 1.125 * sp.sqrt(-8 + t)) * t**3
    LegLHtref10_z = 1/4 * ((10 - t)**(sp.Rational(3, 2)) * (-11 + t) * (-8 + t)**2 + (0.5 - (-8 + t)**(sp.Rational(3, 2))) * (-10 + t)**2 * (-7 + t))

    # Updated Piecewise function construction with all segments
    x = sp.Piecewise((LegLHtref1_x, t <= 1), (LegLHtref3_x, t <= 3), (LegLHtref5_x, t <= 5), 
                     (LegLHtref6_x, t <= 6), (LegLHtref8_x, t <= 8), (LegLHtref10_x, True))
    y = sp.Piecewise((LegLHtref1_y, t <= 1), (LegLHtref3_y, t <= 3), (LegLHtref5_y, t <= 5),
                     (LegLHtref6_y, t <= 6), (LegLHtref8_y, t <= 8), (LegLHtref10_y, True))
    z = sp.Piecewise((LegLHtref1_z, t <= 1), (LegLHtref3_z, t <= 3), (LegLHtref5_z, t <= 5),
                     (LegLHtref6_z, t <= 6), (LegLHtref8_z, t <= 8), (LegLHtref10_z, True))

    # Swap x and y in the return, as per instruction
    return x, y, z


In [5]:
plot = k3d.plot()
plot = knoty.plot_knot(plot, LNGLegTrefoil, domain=[0,8], resolution=500)
plot = knoty.plot_planes_along_knot(plot, LNGLegTrefoil, domain=[0,8], n = 30, size=0.2, form=['y', 0, -1])

plot.display()

/Users/slaus/Library/Python/3.9/lib/python/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "object" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [6]:
plot = k3d.plot()
plot = knoty.plot_knot(plot, LNGLegLHTrefoil, domain=[0,10], resolution=500)
plot = knoty.plot_planes_along_knot(plot, LNGLegLHTrefoil, domain=[0,10], n = 80, form=['y', 0, -1], size=0.1)

plot.display()

Output()

In [7]:

plot = k3d.plot()
plot = knoty.plot_knot(plot, knot2, domain=[0,8], resolution=500)
plot = knoty.plot_planes_along_knot(plot, knot2, n = 30, size=0.1)
plot = knoty.plot_contact_structure(plot, alpha = 0.1)

plot.display()

NameError: name 'np' is not defined

In [ ]:
import knoty
import k3d
import sympy as sp

# Load and process the SVG file
svg_file = './Examples/Blatt4A21.svg'  # Replace with your SVG file path

fun = knoty.import_from_svg(svg_file, scaling=False, threshold=0)

plot = k3d.plot()
plot = knoty.plot_knot(plot, fun, resolution=100, domain=[0,1])
# plot = knoty.plot_planes_along_knot(plot, fun, n = 5, size=0.2)

plot.display()

/Users/slaus/Library/Python/3.9/lib/python/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "object" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()